# Classification notebook
### No feature extraction here - will load from pickle file

In [1]:
import pickle
import numpy as np
import pandas as pd
import json
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
import xgboost
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score, f1_score, roc_curve

In [2]:
def rand_emot():
    e = ["(o_o)",":-)",":P",":D","x)","ᓚᘏᗢ","╯°□°）╯︵ ┻━┻",":)",
         "*<:-)","^_^","(⌐■_■)","¯\_(ツ)_/¯", "(T_T)",":o","OwO",
        "( ͡❛ ͜ʖ ͡❛)","(̶◉͛‿◉̶)","( ≖.≖)","(ㆆ_ㆆ)","ʕ•́ᴥ•̀ʔっ","( ◡́.◡̀)","(^◡^ )"]
    return random.choice(e)

def load_files():
    text_pairs = [] #Would be nice to have as np.array
    labels = []
    fandom = []
    
    pair_id = []
    true_id = []
    
    #Load truth JSON
    for line in open('data/modified/train_truth.jsonl'):
        d = json.loads(line.strip())
        labels.append(int(d['same']))
        true_id.append(d['id'])

    #Load actual fanfic.
    print("loading fanfic...",rand_emot())
    for line in tqdm(open('data/modified/train_pair.jsonl')):
        d = json.loads(line.strip())
        text_pairs.append(d['pair'])
        fandom.append(d['fandoms'])
        pair_id.append(d['id'])

    print("done loading",rand_emot())
    
    return text_pairs, labels, fandom, pair_id, true_id

In [3]:
_, labels, _, _, _ = load_files()    #11550
labels.pop(11550)

307it [00:00, 3047.72it/s]

loading fanfic... (ㆆ_ㆆ)


15780it [00:05, 2978.89it/s]

done loading (o_o)


1

In [4]:
def save_features(feature_dict, filename='features'):
    '''Save the updated feature dictionary. Takes dictionary as input and saves as binary file
    
    example: 
    >>> my_featues = {'freqdist': [1,6,3,5]}
    >>> save_features(my_features)'''
    
    with open('data/{}.dat'.format(filename), 'wb') as file:
        pickle.dump(feature_dict, file)
    print("Features saved! :-)")

def load_features(filename='features'):
    '''Load feature dictionary. Returns the saved feature as a dictionary.
    
    example: 
    >>> my_features = load_features()'''
    
    with open('data/{}.dat'.format(filename), 'rb') as file:
        feats = pickle.load(file)
    print("Features available:")
    for i in feats.keys():
        print(i)
    return feats

In [5]:
feats = load_features(filename="big_features_yasmin")

Features available:
character_bigram
skip_bigram
pos_skipgram
pos_bigram
character_bigram_cossim
skip_bigram_cossim
pos_skipgram_cossim
pos_bigram_cossim
function_word_dist
profanity_dist
unique_words
punctuation_proportion
avg_word_length
avg_sent_length
yules
lix


In [6]:
for i in feats:
    if len(feats[i].shape)==1:
        feats[i] = feats[i][:,None]
    print(i, "\t", feats[i].shape)


character_bigram 	 (15779, 6000)
skip_bigram 	 (15779, 8000)
pos_skipgram 	 (15779, 2242)
pos_bigram 	 (15779, 6000)
character_bigram_cossim 	 (15779, 1)
skip_bigram_cossim 	 (15779, 1)
pos_skipgram_cossim 	 (15779, 1)
pos_bigram_cossim 	 (15779, 1)
function_word_dist 	 (15779, 1)
profanity_dist 	 (15779, 1)
unique_words 	 (15779, 1)
punctuation_proportion 	 (15779, 1)
avg_word_length 	 (15779, 1)
avg_sent_length 	 (15779, 1)
yules 	 (15779, 1)
lix 	 (15779, 1)


## Stratified K-Fold

In [7]:
def strat_kfold(model, X, k):
    y = np.array(labels)
    targets = []
    preds = []
    preds_proba = []
    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=69)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train)
        preds.append(model.predict(X_test))
        preds_proba.append(model.predict_proba(X_test))
        targets.append(y_test)
    
    return np.array([item for elem in preds for item in elem]), np.array([item for elem in targets for item in elem]), np.array([item for elem in preds_proba for item in elem])

def combine_features(features):
    #List of features as input
    return np.hstack((features))

In [8]:
#Stratified 2-Fold accuracy on ALL Features
all_features = combine_features([feats[x] for x in feats])
scalar_features = combine_features([feats[x] for x in feats if feats[x].shape[1]==1])
forced_scalars = []
for i in feats:
    if feats[i].shape[1] >1:
        forced_scalars.append(np.mean(feats[i], axis=1)[:,None])
    else:
        forced_scalars.append(feats[i])

forced_scalars = combine_features(forced_scalars)
vector_features = combine_features([feats['character_bigram'],
                                  feats['skip_bigram'],
                                  feats['pos_skipgram']])

final_features = combine_features([feats['function_word_dist'],
                                 feats['profanity_dist'],
                                 feats['avg_sent_length'],
                                 feats['avg_word_length'],
                                 feats['lix'],
                                 feats['yules'],
                                 feats['unique_words'],
                                 feats['punctuation_proportion'],
                                 feats['character_bigram_cossim'],
                                 feats['skip_bigram_cossim'],
                                 feats['pos_skipgram_cossim'],
                                 feats['pos_bigram_cossim']])

print("All features:",all_features.shape)
print("Scalar features:", scalar_features.shape)
print("Forced scalar features:", forced_scalars.shape)
print("Vector features:", vector_features.shape)
print("Final features:", final_features.shape)

All features: (15779, 22254)
Scalar features: (15779, 12)
Forced scalar features: (15779, 16)
Vector features: (15779, 16242)
Final features: (15779, 12)


In [9]:
def try_different_feats(model):
    all_preds, all_targs, all_pred_proba = strat_kfold(model, all_features, 5)
    vec_preds, vec_targs, vec_pred_proba = strat_kfold(model, vector_features, 5)
    fin_preds, fin_targs, fin_pred_proba = strat_kfold(model, final_features, 5)
    
    print("All features: F1={}\tAcc={}".format(f1_score(all_targs, all_preds), accuracy_score(all_targs, all_preds)))
    print("Vector features: F1={}\tAcc={}".format(f1_score(vec_targs, vec_preds), accuracy_score(vec_targs, vec_preds)))
    print("Final features: F1={}\tAcc={}".format(f1_score(fin_targs, fin_preds), accuracy_score(fin_targs, fin_preds)))
    
    return [(all_pred_proba, all_targs), (vec_pred_proba, vec_targs), (fin_pred_proba, fin_targs)]

## Random Forest

In [10]:
rfc = RandomForestClassifier(n_estimators=500, random_state=42, n_jobs=4)
rfc_results = try_different_feats(rfc)

All features: F1=0.8252336448598131	Acc=0.8103808859877052
Vector features: F1=0.7611021069692058	Acc=0.7197541035553584
Final features: F1=0.7999257379788354	Acc=0.7951074212560999


In [11]:
#Stratified 5-Fold on the 8k best Random Forest Features
#rfc_8k_feats = all_features[:,rfc.feature_importances_.argsort()[-8000:][::-1]]
#strat_kfold(rfc, rfc_8k_feats, 5)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(final_features, labels)

dist = {
    'n_estimators' : [100,200,300,500,1000],
    'min_samples_split': [2,4,6,8],
    'min_samples_leaf': [1,2,3,4,5],
}
random_search = RandomizedSearchCV(rfc, dist, n_iter=20, random_state=42)
search = random_search.fit(X_train, y_train)
params = search.best_params_
search.score(X_test, y_test)

0.7987325728770596

In [13]:
strat_kfold(search, final_features, 5)

(array([0, 0, 1, ..., 0, 0, 1]),
 array([0, 0, 0, ..., 0, 0, 1]),
 array([[0.73931614, 0.26068386],
        [0.83187963, 0.16812037],
        [0.01561111, 0.98438889],
        ...,
        [0.88373137, 0.11626863],
        [0.75604036, 0.24395964],
        [0.2008957 , 0.7991043 ]]))

## XGBoost

In [14]:
xgb = xgboost.XGBClassifier(random_state=42)
xgb_results = try_different_feats(xgb)

c:\python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:38:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:49:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:00:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:11:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

c:\python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[22:32:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:41:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:49:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[22:58:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

c:\python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[23:14:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:14:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:14:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[23:14:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [15]:
X_train, X_test, y_train, y_test = train_test_split(final_features, labels, random_state=42)

dist = {
    'n_estimators' : [100,200,300,500,1000,2000],
    'max_depth' : [6,7,8,9,10],
    'gamma' : [0,1,2,3,5,8],
    'learning_rate':[0.003, 0.01, 0.03, 0.1],
    "eval_metric": ["logloss", "error"],
}
random_search = RandomizedSearchCV(xgb, dist, n_iter=20, random_state=42)
search = random_search.fit(X_train, y_train)
params = search.best_params_
search.score(X_test, y_test)

c:\python37\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.7934093789607097

## Support Vector Machine

In [ ]:
svm_clf = svm.SVC(probability=True, random_state=42)
svc_results = try_different_feats(svm_clf)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(final_features, labels)

dist = {
    'C' : [1,2,3],
    'gamma' : ['scale', 'auto'],
}
random_search = RandomizedSearchCV(svm_clf, dist, n_iter=20, random_state=42)
search = random_search.fit(X_train, y_train)
params = search.best_params_
search.score(X_test, y_test)

## Logistic Regression

In [ ]:
lrg = LogisticRegression(max_iter=2000, n_jobs=4, random_state=42)
lrg_results = try_different_feats(lrg)

## Naive Bayes

In [ ]:
nb = GaussianNB(random_state=42)
nb_results = try_different_feats(nb)

## MLP Test

In [ ]:
mlp = MLPClassifier(max_iter=1000, random_state=42)
mlp_results = try_different_feats(mlp)

## Dummy Baseline

In [ ]:
dummy = DummyClassifier(random_state=42)
dummy_results = try_different_feats(dummy)

## ROC Curve

In [ ]:
def plot_roc_cur(fper, tper, label="", title="plot title"):
    plt.plot(fper, tper, label=label)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc='lower right')
    
def create_roc_curve(preds, targets, label="", title=""):
    all_fpr, all_tpr = [], []
    print(preds[:,1])
    fpr, tpr, thresholds = roc_curve(targets, preds[:,1]) #De her burde vist være nyttet om...
    all_fpr.append(fpr)
    all_tpr.append(tpr)
    all_fpr = np.array(all_fpr)
    all_tpr = np.array(tpr)
    
    plot_roc_cur(all_fpr[0], all_tpr, label=label, title=title)

In [ ]:
labels = ['Random Forest', 'Support Vector Machine', 'Logistic Regression', 'XGBoost', 'Multilayer Perceptron']
all_results = [rfc_results[0][0], svc_results[0][0], lrg_results[0][0], xgb_results[0][0], mlp_results[0][0]]
all_targets = [rfc_results[0][1], svc_results[0][1], lrg_results[0][1], xgb_results[0][1], mlp_results[0][1]]

vec_results = [rfc_results[1][0], svc_results[1][0], lrg_results[1][0], xgb_results[1][0], mlp_results[1][0]]
vec_targets = [rfc_results[1][1], svc_results[1][1], lrg_results[1][1], xgb_results[1][1], mlp_results[1][1]]

fin_results = [rfc_results[2][0], svc_results[2][0], lrg_results[2][0], xgb_results[2][0], mlp_results[2][0]]
fin_targets = [rfc_results[2][1], svc_results[2][1], lrg_results[2][1], xgb_results[2][1], mlp_results[2][1]]

In [ ]:
for i in range(len(all_results)):
    create_roc_curve(all_results[i], all_targets[i], label=labels[i], title="ROC Curve For All Features")

In [ ]:
for i in range(len(vec_results)):
    create_roc_curve(vec_results[i], vec_targets[i], label=labels[i], title="ROC Curve For Vector Features")

In [ ]:
for i in range(len(fin_results)):
    create_roc_curve(fin_results[i], fin_targets[i], label=labels[i], title="ROC Curve For Scalar Features")